# Algorytm 1

In [16]:
def naiveNonFilter(X, criteria):
    P = []
    number_of_comparisons = 0
    l = len(X)

    for i in range(l):
        X_temp = X.copy()
        Y = X[0]
        l = len(X)
        fl = 0
        for j in range(1, l):
            if j >= len(X):
                break
            number_of_comparisons += 2
            Y_dominates_Xj = all(
                Y[k] <= X[j][k] if criteria[k] == 'min' else Y[k] >= X[j][k]
                for k in range(len(Y))
            )
            Xj_dominates_Y = all(
                X[j][k] <= Y[k] if criteria[k] == 'min' else X[j][k] >= Y[k]
                for k in range(len(Y))
            )
            if Y_dominates_Xj:
                X_temp.remove(X[j])
            elif Xj_dominates_Y:
                Y_temp = X[j]
                X_temp.remove(Y)
                Y = Y_temp
                fl = 1
        if Y not in P:
            P.append(Y)
        if fl == 0:
            X_temp.remove(Y)
        if len(X_temp) == 1:
            if X_temp[0] not in P:
                P.append(X_temp[0])
            break
        X = X_temp.copy()
    return P, number_of_comparisons


# X = [(5, 5), (3, 6), (4, 4), (5, 3), (3, 3), (1, 8), (3, 4), (4, 5), (3, 10), (6, 6), (4, 1), (3, 5)]
# criteria = ['min', 'min'] 

X = [
    (5, 5, 7),
    (3, 6, 8),
    (4, 4, 6),
    (5, 3, 5),
    (3, 3, 3),
    (1, 8, 9),
    (3, 4, 4),
    (4, 5, 7),
    (3, 10, 12),
    (6, 6, 6),
    (4, 1, 2),
    (3, 5, 5)
]

# Kryteria: minimalizacja pierwszych dwóch, maksymalizacja trzeciego
criteria = ['min', 'min', 'max']


P, no = naiveNonFilter(X, criteria)
print("Punkty Pareto:", P)
print("Liczba porównań:", no)

Punkty Pareto: [(4, 5, 7), (3, 6, 8), (4, 4, 6), (5, 3, 5), (3, 3, 3), (1, 8, 9), (3, 4, 4), (3, 10, 12), (4, 1, 2), (3, 5, 5)]
Liczba porównań: 112


# Algorytm 2

In [19]:
def naiveWithFilter(X, criteria):
    P = []
    number_of_comparisons = 0

    while X:
        Y = X[0]
        dominated_by_Y = []
        dominates_Y = False
        for x_j in X[1:]:
            number_of_comparisons += 2
            Y_dominates_Xj = all(
                Y[k] <= x_j[k] if criteria[k] == 'min' else Y[k] >= x_j[k]
                for k in range(len(Y))
            )
            Xj_dominates_Y = all(
                x_j[k] <= Y[k] if criteria[k] == 'min' else x_j[k] >= Y[k]
                for k in range(len(Y))
            )
            if Y_dominates_Xj:
                dominated_by_Y.append(x_j)
            elif Xj_dominates_Y:
                Y = x_j
                dominates_Y = True
                break  # Restartujemy z nowym Y
        if dominates_Y:
            X.remove(Y)
            continue  # Restartujemy pętlę z nowym Y
        else:
            P.append(Y)
            X.remove(Y)
            for x in dominated_by_Y:
                if x in X:
                    X.remove(x)
        # Usuwamy punkty zdominowane przez Y
        to_remove = []
        for x_k in X:
            number_of_comparisons += 2
            Y_dominates_Xk = all(
                Y[k] <= x_k[k] if criteria[k] == 'min' else Y[k] >= x_k[k]
                for k in range(len(Y))
            )
            if Y_dominates_Xk:
                to_remove.append(x_k)
        for x in to_remove:
            if x in X:
                X.remove(x)
        if len(X) == 1:
            P.append(X[0])
            break
    return P, number_of_comparisons

# Dane z trzema kryteriami
# X = [
#     (5, 5, 7),
#     (3, 6, 8),
#     (4, 4, 6),
#     (5, 3, 5),
#     (3, 3, 3),
#     (1, 8, 9),
#     (3, 4, 4),
#     (4, 5, 7),
#     (3, 10, 12),
#     (6, 6, 6),
#     (4, 1, 2),
#     (3, 5, 5)
# ]

# # Kryteria: minimalizacja pierwszych dwóch, maksymalizacja trzeciego
# criteria = ['min', 'min', 'max']

# P, no = naiveWithFilter(X, criteria)

# print(f"Liczba porównań: {no}")
# print("Punkty Pareto:")
# for p in P:
#     print(p)


X = [(5, 5), (3, 6), (4, 4), (5, 3), (3, 3), (1, 8), (3, 4), (4, 5), (3, 10), (6, 6), (4, 1), (3, 5)]
criteria = ['min', 'min']  # Minimalizacja pierwszego kryterium, maksymalizacja drugiego

P, no = naiveWithFilter(X, criteria)

print(f"Liczba porównań: {no}")
print("Punkty Pareto:")
for p in P:
    print(p)


Liczba porównań: 64
Punkty Pareto:
(5, 5)
(3, 6)
(1, 8)


# Algorytm 3

In [21]:
import math

def distance(p1, p2):
    return math.sqrt(sum((p1[k] - p2[k]) ** 2 for k in range(len(p1))))

def calculate_ideal_point(X, criteria):
    ideal = []
    for k in range(len(X[0])):
        if criteria[k] == 'min':
            ideal.append(min(point[k] for point in X))
        else:
            ideal.append(max(point[k] for point in X))
    return tuple(ideal)

def dominates(p1, p2, criteria):
    better_in_all = True
    better_in_at_least_one = False
    for k in range(len(p1)):
        if criteria[k] == 'min':
            if p1[k] > p2[k]:
                better_in_all = False
                break
            elif p1[k] < p2[k]:
                better_in_at_least_one = True
        else:
            if p1[k] < p2[k]:
                better_in_all = False
                break
            elif p1[k] > p2[k]:
                better_in_at_least_one = True
    return better_in_all and better_in_at_least_one

def naiveIdealPoint(X, criteria):
    ideal = calculate_ideal_point(X, criteria)

    distances = [(distance(p, ideal), idx) for idx, p in enumerate(X)]
    distances.sort(key=lambda x: x[0])

    sorted_indices = [index for _, index in distances]

    P = []
    number_of_comparisons = 0
    X_temp = X.copy()
    m = 0

    while len(X_temp) > 0 and m < len(sorted_indices):
        current_p = X[sorted_indices[m]]
        if current_p in X_temp:
            P.append(current_p)
            X_temp.remove(current_p)
        
            dominated_p = []
            for p in X_temp:
                number_of_comparisons += 1
                if dominates(current_p, p, criteria):
                    dominated_p.append(p)
            
            for dp in dominated_p:
                X_temp.remove(dp)
        
        m += 1
    
    return P, number_of_comparisons 



X = [(5, 5), (3, 6), (4, 4), (5, 3), (3, 3), (1, 8), (3, 4), (4, 5), (3, 10), (6, 6), (4, 1), (3, 5)]
criteria = ['min', 'min']  # Minimalizacja obu kryteriów

P, no = naiveIdealPoint(X, criteria)

print(f"Liczba porównań: {no}")
print("Punkty Pareto:")
for p in P:
    print(p)


Liczba porównań: 12
Punkty Pareto:
(3, 3)
(4, 1)
(1, 8)
